In [1]:
import pandas as pd
import numpy as np
import math as mt
import streamlit as st
from scipy import stats as st

In [2]:
# Import Data
vehicles = pd.read_csv('vehicles_us.csv')

In [3]:
# EDA
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


**Initial Observations**
- `Model_year`, `cylinders`, `odometer`, `paint_color`, and `is_4wd` have null values.
-All null value columns except `is_4wd` can be filled with N/A. `is_4wd` can be filled with zero to indicate no 4-wheel drive
- `Model` column currently contains the make and model of the car in the same value. Would be better to split this in to `Make` and `Model` columns so better analysis can be performed.

In [4]:
vehicles.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


In [5]:
# Fill null values with "N/A" or other appropriate filler.
vehicles['model_year'] = vehicles['model_year'].fillna('N/A')
vehicles['model_year'].head(10)

0    2011.0
1       N/A
2    2013.0
3    2003.0
4    2017.0
5    2014.0
6    2015.0
7    2013.0
8    2012.0
9    2008.0
Name: model_year, dtype: object

In [6]:
# Rename model column to make_model to prepare for change
vehicles = vehicles.rename(columns={'model':'make_model'})
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  object 
 2   make_model    51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 5.1+ MB


In [7]:
#Check cell for comparison later
vehicles['make_model'].head(10)

0            bmw x5
1        ford f-150
2    hyundai sonata
3        ford f-150
4      chrysler 200
5      chrysler 300
6      toyota camry
7       honda pilot
8       kia sorento
9       honda pilot
Name: make_model, dtype: object

In [8]:
# Get unique values from the model category
vehicles['make_model'].unique()

array(['bmw x5', 'ford f-150', 'hyundai sonata', 'chrysler 200',
       'chrysler 300', 'toyota camry', 'honda pilot', 'kia sorento',
       'chevrolet silverado 1500', 'honda accord', 'ram 1500',
       'gmc yukon', 'jeep cherokee', 'chevrolet traverse',
       'hyundai elantra', 'chevrolet tahoe', 'toyota rav4',
       'chevrolet silverado', 'jeep wrangler', 'chevrolet malibu',
       'ford fusion se', 'chevrolet impala', 'chevrolet corvette',
       'jeep liberty', 'toyota camry le', 'nissan altima',
       'subaru outback', 'toyota highlander', 'dodge charger',
       'toyota tacoma', 'chevrolet equinox', 'nissan rogue',
       'mercedes-benz benze sprinter 2500', 'honda cr-v',
       'jeep grand cherokee', 'toyota 4runner', 'ford focus',
       'honda civic', 'kia soul', 'chevrolet colorado',
       'ford f150 supercrew cab xlt', 'chevrolet camaro lt coupe 2d',
       'chevrolet cruze', 'ford mustang', 'chevrolet silverado 3500hd',
       'nissan frontier crew cab sv', 'subaru imp

In [9]:
# Split items in make_model column and assign them to individual make and model columns
vehicles['make'] = vehicles['make_model'].str.split().str[0]
vehicles['model'] = vehicles['make_model'].str.split().str[1:]
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  object 
 2   make_model    51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  make          51525 non-null  object 
 14  model         51525 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 5.9+ MB


In [10]:
# Above change leaves the model column as a series of lists. Below will concatenate the lists into single spaced strings
vehicles['model'] = vehicles['model'].apply(lambda x:' '.join(x))
vehicles['model'].head(10)

0         x5
1      f-150
2     sonata
3      f-150
4        200
5        300
6      camry
7      pilot
8    sorento
9      pilot
Name: model, dtype: object

In [20]:
# Check `condition` column for irregularities
print(vehicles['condition'].unique())

['good' 'like new' 'fair' 'excellent' 'salvage' 'new']


In [18]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  object 
 2   make_model    51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  object 
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  object 
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  make          51525 non-null  object 
 14  model         51525 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 5.9+ MB


In [15]:
# Fill various null values in columns with N/A
vehicles['cylinders'] = vehicles['cylinders'].fillna('N/A')
vehicles['odometer'] = vehicles['odometer'].fillna('N/A')
vehicles['paint_color'] = vehicles['paint_color'].fillna('N/A')

# Fill null values in is_4wd column with 0.00 to indicate no 4-wheel drive
vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0.00)

#Check for duplicated rows
print(vehicles.duplicated().unique())

#Check vehicle info again
print(vehicles.info())

[False]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  object 
 2   make_model    51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  object 
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  object 
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  make          51525 non-null  object 
 14  model         51525 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 5.9+ MB
None
